# Week 9 Homework

In [1]:
#@markdown What is your Pennkey and pod? (text, not numbers, e.g. bfranklin)
my_pennkey = 'jabrantl' #@param {type:"string"}
my_pod = 'superfluous-lyrebird' #@param ['Select', 'euclidean-wombat', 'sublime-newt', 'buoyant-unicorn', 'lackadaisical-manatee','indelible-stingray','superfluous-lyrebird','discreet-reindeer','quizzical-goldfish','astute-jellyfish','ubiquitous-cheetah','nonchalant-crocodile','fashionable-lemur','spiffy-eagle','electric-emu','quotidian-lion']


## Section 1: Role of social media platforms in mitigating hate speech

Social media platforms have instituted not just user policies for online conduct, but also technological mechanisms to identify and remove hate speech.  As an example, review Facebook’s discussion of how its employing AI to identify hate speech.  As you read this article, pay particular attention to the specific challenges they enumerate in identifying hate speech. 

"AI advances to better detect hate speech" 

https://ai.facebook.com/blog/ai-advances-to-better-detect-hate-speech/

*Facebook, 2020* [10-minute read]

The Facebook article emphasizes that the challenges to identifying hate speech are both social and technological.  The social considerations include the importance of cultural context as well as the nuances of languages. How does that resonate with the "Excavating AI: The Politics of Images in Machine Learning Training Sets" article you read in Week 6?  What do you think of Facebook’s approach? (200-300 words)

In [ ]:
section1_ans = "This problem seems like an almost insurmountable challenge. To me, ~90% accuracy detection of hate speech is quite impressive considering all of the nuance and slang that is often used in hateful speech. Examples of direct threats or hate speech seems somewhat simple to detect, but in many cases, hate speech/writing can occur through difficult to recognize bullying. For example, the idea of combining ambiguous text onto images to create hateful memes seems like an incredibly difficult challenge. The one concern in all of this is how the speech is flagged as 'hateful' in the first place. In fact, many people (particularly those in more conservative circles) often complain about their speech being flagged as hateful and companies overstepping and violating the 1st amendment. However,  hate speech is not protected by the 1st amendment  , so it is right to be removed. On the other hand, the definition of hate speech can be manipulated to reflect the bias of those doing the labeling. I like the idea of self-supervised learning as a way to identify potential hate speech w/o humans in the loop, but this could also lead to accidental suppression of speech that was not actually hate speech or was quoting or rebuttal to hate speech.   The second part of the article is more challenging for me. I see how the discussion about language styles being perceived as 'wrong' in a social media environment is bad, but I still struggle with the idea of having vernacular that uses complete improper grammar as being correct English. I also sometimes struggle with the criticism that you have to \"talk White\" to be accepted as talking correctly. My family speaks English and Spanish and in both languages, grammar and structure are still present and somewhat interchangeable between both. I find the argument about double negatives to be weak unless it purely hinges on the word 'logic'. Maybe illogical is incorrect, but just because something is ok in another language does not mean that it has to hold for a different one. Just because you can do it in Italian does not mean that it is also ok in English. That would also mean that in  " #@param {type:"string"}

## Section 2: Broader societal considerations of addressing hate speech on social media


Next, read this Medium article about the unintended consequences of technological interventions to address hate speech on social media.

"How Automated Tools Discriminate Against Black Language"    
https://medium.com/@annawchung/how-automated-tools-discriminate-against-black-language-2ac8eab8d6db.  
*Medium, 2019* [14-minute read]

"Challenges in Automated Debiasing for Toxic Language Detection"    
https://arxiv.org/pdf/2102.00086.pdf.  
*Optional – this is much longer and more technical!!!*

How does this example illustrate the complexity of identifying hate speech?  What solutions does the author suggest? Why is it important that AI practitioners have a robust understanding of the societal context and consequence of their work? (1-2 paragraphs)

In [ ]:
section2_ans = 'Clearly the method for labeling text as toxic is completely flawed and has huge racial, gender, and sexual identity bias. Although I do not know the technical side of how these expressions are being labeled as toxic, it is not clear to me at all how this could occur in the first place w/o having seriously corrupted training data--i.e., people were labeling things with their own biases being very strong. I find the second part of the article to be less compelling, but perhaps that discussion is for a later time.   ' #@param {type:"string"}

## Section 3: Ablation Study on LSTMs

In the tutorials we learnt about the different gates in LSTMs. We also learnt about GRUs that have fewer components than LSTMs but work reasonably well. In this section of the homework you will conduct an ablation study for LSTMs. You will be working on text generation task. The model will be trained on H. G. Wells’ *The Time Machine*.

In order to have a baseline, first intialize the weights and implement the gates of a LSTM. 

For the ablation study change different components listed below (these are suggestions, you are free to use your own ideas as well) and observe the effects on performance. 
- Initialization
- Number of gates
- Replacing sigmoid, tanh with other functions
- Changing the inputs used for each recurrent unit

You will make changes in the `get_lstm_params` and `lstm` functions. We have provided code for forward and backward propagation, training and a sample code to see the text generation results. 
(Please remember to to return tensors with the same shape as the LSTM you will implement to ensure end to end training.)

In [2]:
#@title Imports
!pip install d2l

import torch
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F
import math

     |████████████████████████████████| 81kB 2.6MB/s 


In [3]:
def get_lstm_params(vocab_size, num_hiddens):
    
    num_inputs = num_outputs = vocab_size

    # i_t: input gate layer
    W_xi = nn.Parameter(torch.Tensor(num_inputs,num_hiddens))
    W_hi = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    b_i  = nn.Parameter(torch.Tensor(num_hiddens))

    # f_t: forget gate
    W_xf = nn.Parameter(torch.Tensor(num_inputs,num_hiddens))
    W_hf = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    b_f  = nn.Parameter(torch.Tensor(num_hiddens))

    # o_t: output gate
    W_xo = nn.Parameter(torch.Tensor(num_inputs,num_hiddens))
    W_ho = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    b_o  = nn.Parameter(torch.Tensor(num_hiddens))

    # c_t: Cell state
    W_xc = nn.Parameter(torch.Tensor(num_inputs,num_hiddens))
    W_hc = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    b_c  = nn.Parameter(torch.Tensor(num_hiddens))

    # FCN from hidden --> output
    W_hq = nn.Parameter(torch.Tensor(num_hiddens,num_outputs))
    b_q  = nn.Parameter(torch.Tensor(num_outputs))
 
    # compute values for initialization
    sigma = 1.0 / math.sqrt(num_hiddens)
    
    # List all the parameters and attach gradients
    params = [W_xi, W_hi, b_i, 
              W_xf, W_hf, b_f, 
              W_xo, W_ho, b_o, 
              W_xc, W_hc, b_c,
              W_hq, b_q]
    for param in params:
        # Initialize weights using xavier 
        param.data.uniform_(-sigma,sigma)
        # Requre grad = true
        param.requires_grad_(True)

    return params

In [4]:
def lstm(inputs, state, params):
    # Get the model params
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c,
        W_hq, b_q] = params
    # Get the hidden and cell state
    (H, C) = state

    # Store outputs for each input from a batch 
    outputs = []

    # For each of the inputs in a batch, perform the LSTM operations
    for X in inputs:
      # Input gate
      i_t = torch.sigmoid(X @ W_xi + H @ W_hi + b_i)
      # Forget gate
      f_t = torch.sigmoid(X @ W_xf + H @ W_hf + b_f)
      # Compute c_tilda
      c_tilda = torch.tanh(X @ W_xc + H @ W_hc + b_c)
      # New cell state C_t = old cell state * f_t + i_t * c_tilda
      C_t = (f_t * C) + (i_t * c_tilda)
      # Output gate
      o_t = torch.sigmoid(X @ W_xo + H @ W_ho + b_o)
      # Compute new hidden state
      h_t = o_t * torch.tanh(C_t)
      
      # Update cell and hidden state
      H, C = h_t, C_t

      # Append output by passing hidden through fully connected output
      outputs.append(h_t @ W_hq + b_q)

    return torch.cat(outputs, dim=0), (H, C)

In [ ]:
def get_lstm_peephole_params(vocab_size, num_hiddens):
    
    num_inputs = num_outputs = vocab_size

    # i_t: input gate layer
    W_xi = nn.Parameter(torch.Tensor(num_inputs,num_hiddens))
    W_hi = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    W_ci = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    b_i  = nn.Parameter(torch.Tensor(num_hiddens))

    # f_t: forget gate
    W_xf = nn.Parameter(torch.Tensor(num_inputs,num_hiddens))
    W_hf = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    W_cf = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    b_f  = nn.Parameter(torch.Tensor(num_hiddens))

    # o_t: output gate
    W_xo = nn.Parameter(torch.Tensor(num_inputs,num_hiddens))
    W_ho = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    W_co = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    b_o  = nn.Parameter(torch.Tensor(num_hiddens))

    # c_t: Cell state
    W_xc = nn.Parameter(torch.Tensor(num_inputs,num_hiddens))
    W_hc = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    b_c  = nn.Parameter(torch.Tensor(num_hiddens))

    # FCN from hidden --> output
    W_hq = nn.Parameter(torch.Tensor(num_hiddens,num_outputs))
    b_q  = nn.Parameter(torch.Tensor(num_outputs))
 
    # compute values for initialization
    sigma = 1.0 / math.sqrt(num_hiddens)
    
    # List all the parameters and attach gradients
    params = [W_xi, W_hi, W_ci, b_i, 
              W_xf, W_hf, W_cf, b_f, 
              W_xo, W_ho, W_co, b_o, 
              W_xc, W_hc, b_c,
              W_hq, b_q]
    for param in params:
        # Initialize weights using xavier 
        param.data.uniform_(-sigma,sigma)
        # Requre grad = true
        param.requires_grad_(True)

    return params

def lstm_peephole(inputs, state, params):
    # Get the model params
    [W_xi, W_hi, W_ci, b_i, W_xf, W_hf, W_cf, b_f, W_xo, W_ho, W_co, b_o, W_xc, W_hc, b_c,
        W_hq, b_q] = params
    # Get the hidden and cell state
    (H, C) = state

    # Store outputs for each input from a batch 
    outputs = []

    # For each of the inputs in a batch, perform the LSTM operations
    for X in inputs:
      # Input gate
      i_t = torch.sigmoid(X @ W_xi + C @ W_ci + H @ W_hi + b_i)
      # Forget gate
      f_t = torch.sigmoid(X @ W_xf + C @ W_cf + H @ W_hf + b_f)
      # Compute c_tilda
      c_tilda = torch.tanh(X @ W_xc + C @ W_co + H @ W_hc + b_c)
      # New cell state C_t = old cell state * f_t + i_t * c_tilda
      C_t = (f_t * C) + (i_t * c_tilda)
      # Output gate
      o_t = torch.sigmoid(X @ W_xo + H @ W_ho + b_o)
      # Compute new hidden state
      h_t = o_t * torch.tanh(C_t)
      
      # Update cell and hidden state
      H, C = h_t, C_t

      # Append output by passing hidden through fully connected output
      outputs.append(h_t @ W_hq + b_q)

    return torch.cat(outputs, dim=0), (H, C) 

In [5]:
def get_lstm_coupleForgetInput_params(vocab_size, num_hiddens):
    
    num_inputs = num_outputs = vocab_size

    # # i_t: input gate layer
    # W_xi = nn.Parameter(torch.Tensor(num_inputs,num_hiddens))
    # W_hi = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    # b_i  = nn.Parameter(torch.Tensor(num_hiddens))

    # f_t: forget gate
    W_xf = nn.Parameter(torch.Tensor(num_inputs,num_hiddens))
    W_hf = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    b_f  = nn.Parameter(torch.Tensor(num_hiddens))

    # o_t: output gate
    W_xo = nn.Parameter(torch.Tensor(num_inputs,num_hiddens))
    W_ho = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    b_o  = nn.Parameter(torch.Tensor(num_hiddens))

    # c_t: Cell state
    W_xc = nn.Parameter(torch.Tensor(num_inputs,num_hiddens))
    W_hc = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    b_c  = nn.Parameter(torch.Tensor(num_hiddens))

    # FCN from hidden --> output
    W_hq = nn.Parameter(torch.Tensor(num_hiddens,num_outputs))
    b_q  = nn.Parameter(torch.Tensor(num_outputs))
 
    # compute values for initialization
    sigma = 1.0 / math.sqrt(num_hiddens)
    
    # List all the parameters and attach gradients
    params = [W_xf, W_hf, b_f, 
              W_xo, W_ho, b_o, 
              W_xc, W_hc, b_c,
              W_hq, b_q]
    for param in params:
        # Initialize weights using xavier 
        param.data.uniform_(-sigma,sigma)
        # Requre grad = true
        param.requires_grad_(True)

    return params

def lstm_coupleForgetInput(inputs, state, params):
    # Get the model params
    [W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c,
        W_hq, b_q] = params
    # Get the hidden and cell state
    (H, C) = state

    # Store outputs for each input from a batch 
    outputs = []

    # For each of the inputs in a batch, perform the LSTM operations
    for X in inputs:
      # Input gate: not used when coupling input/forget
      # i_t = torch.sigmoid(X @ W_xi + H @ W_hi + b_i)
      # Forget gate
      f_t = torch.sigmoid(X @ W_xf + H @ W_hf + b_f)
      # Compute c_tilda
      c_tilda = torch.tanh(X @ W_xc + H @ W_hc + b_c)
      # New cell state C_t = old cell state * f_t + i_t * c_tilda
      C_t = (f_t * C) + ((torch.ones_like(f_t) - f_t) * c_tilda)
      # Output gate
      o_t = torch.sigmoid(X @ W_xo + H @ W_ho + b_o)
      # Compute new hidden state
      h_t = o_t * torch.tanh(C_t)
      
      # Update cell and hidden state
      H, C = h_t, C_t

      # Append output by passing hidden through fully connected output
      outputs.append(h_t @ W_hq + b_q)

    return torch.cat(outputs, dim=0), (H, C) 
  

In [6]:
def get_gru_params(vocab_size, num_hiddens):
    
    num_inputs = num_outputs = vocab_size

    # z_t
    W_xz = nn.Parameter(torch.Tensor(num_inputs,num_hiddens))
    W_hz = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))

    # r_t
    W_xr = nn.Parameter(torch.Tensor(num_inputs,num_hiddens))
    W_hr = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    
    # W in h_tilda
    W_htildax = nn.Parameter(torch.Tensor(num_inputs,num_hiddens))
    W_htildar = nn.Parameter(torch.Tensor(num_hiddens,num_hiddens))
    
    # FCN from hidden --> output
    W_hq = nn.Parameter(torch.Tensor(num_hiddens,num_outputs))
    b_q  = nn.Parameter(torch.Tensor(num_outputs))
 
    # compute values for initialization
    sigma = 1.0 / math.sqrt(num_hiddens)
    
    # List all the parameters and attach gradients
    params = [W_xz, W_hz, 
              W_xr, W_hr, 
              W_htildax, W_htildar,
              W_hq, b_q]
    for param in params:
        # Initialize weights using xavier 
        param.data.uniform_(-sigma,sigma)
        # Requre grad = true
        param.requires_grad_(True)

    return params

def gru(inputs, state, params):
    # Get the model params
    [W_xz, W_hz, W_xr, W_hr, W_htildax, W_htildar, W_hq, b_q] = params
    
    # Get the hidden and cell state - cell state not used for GRU but easier to leave than edit all code
    (H, C) = state

    # Store outputs for each input from a batch 
    outputs = []

    # For each of the inputs in a batch, perform the LSTM operations
    for X in inputs:
      # z_t
      z_t = torch.sigmoid(X @ W_xz + H @ W_hz)
      # r_t
      r_t = torch.sigmoid(X @ W_xr + H @ W_hr )
      # Compute h_tilda
      h_tilda = torch.tanh(X @ W_htildax + (H * r_t) @ W_htildar)
      # Compute h_t
      h_t = (torch.ones_like(z_t) - z_t) * H + z_t * h_tilda
      # Update cell and hidden state
      H = h_t

      
      
      # Append output by passing hidden through fully connected output
      outputs.append(h_t @ W_hq + b_q)

    return torch.cat(outputs, dim=0), (H, C) 

In [7]:
# Build the LSTM model from scratch
class LSTMScratch:  
    
    """An LSTM Model implemented from scratch."""
    
    def __init__(self, vocab_size, num_hiddens, get_params, forward_fn):

        self.vocab_size, self.num_hiddens, self.forward_fn = vocab_size, num_hiddens, forward_fn
        # Function to initialize all the parameters in the LSTM (defined by you!)
        self.params = get_params(vocab_size, num_hiddens)

    def __call__(self, X, state):
        # Calls the forward function 
        
        # Convert X to a one-hot vector for each character instead of an Embedding
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        
        # Function to pass inputs to the model (defined by you!)
        return self.forward_fn(X, state, self.params)

    def begin_state(self, batch_size):
        # Initialize the hidden state and cell state with zeros
        return (torch.zeros((batch_size, num_hiddens), device=device),
            torch.zeros((batch_size, num_hiddens), device=device))

In [8]:
#@title Training Functions (Run Me!)
# Clip the gradients
def grad_clipping(net, theta):
    """Clip the gradient."""
    params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad**2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

# Optimization Minibatch SGD
def sgd(params, lr, batch_size):
    """Minibatch stochastic gradient descent."""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

# Training in each epoch
def train_epoch(net, train_iter, loss, optimizer):
    """Train a net within one epoch (defined in Chapter 8)."""

    state, timer = None, d2l.Timer()
    metric = d2l.Accumulator(2)  # Sum of training loss, no. of tokens
    
    # For every barch in dataloader
    for X, Y in train_iter:
        if state is None:
            # Initialize `state` when either it is the first iteration or
            # using random sampling
            state = net.begin_state(batch_size=X.shape[0])
        else:
            # `state` is a tuple of tensors for our custom scratch implementation
            for s in state:
                s.detach_()

        # Pass input and state to the model
        y = Y.T.reshape(-1)
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, state)

        # Compute loss, backpropagate and clip the gradient
        l = loss(y_hat, y.long()).mean()
        l.backward()
        grad_clipping(net, 1)

        # Step the optimizer
        optimizer(batch_size=1)

        # Loss metric append    
        metric.add(l * y.numel(), y.numel())
    return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()

# Predict function
def predict(prefix, num_preds, net, vocab):  
    """Generate new characters following the `prefix`."""    
    # Initialize and tensorize input
    state = net.begin_state(batch_size=1)
    outputs = [vocab[prefix[0]]]
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape(
        (1, 1))
    
    for y in prefix[1:]:  # Warm-up period
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    
    for _ in range(num_preds):  # Predict `num_preds` steps
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    
    return ''.join([vocab.idx_to_token[i] for i in outputs])

# Train function
def train(net, train_iter, vocab, lr, num_epochs):
    """Train a model."""

    # Loss function and loss visualization tool 
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', ylabel='perplexity', legend=['train'], xlim=[10, num_epochs])
    
    # Optimizer
    optimizer = lambda batch_size: sgd(net.params, lr, batch_size)

    # Train and predict over each epoch
    for epoch in range(num_epochs):
        ppl, speed = train_epoch(net, train_iter, loss, optimizer)
        
        # Plot every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(predict('time traveller', 50, net, vocab))
            animator.add(epoch + 1, [ppl])
    
    # Final predictions
    print(f'perplexity {ppl:.1f}, {speed:.1f} tokens/sec on {str(device)}')
    print(predict('time traveller', 50, net, vocab))
    print(predict('traveller', 50, net, vocab))

In [9]:
batch_size, num_steps = 32, 50

train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

In [ ]:
train_iter.__dict__

In [12]:
vocab_size, num_hiddens, device = len(vocab), 256, torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs, lr = 500, 1

model = LSTMScratch(len(vocab), num_hiddens, get_gru_params, gru)
train(model, train_iter, vocab, lr, num_epochs)

In [14]:
print(model.params)

[Parameter containing:
tensor([[-0.0565, -0.0284,  0.0555,  ..., -0.0371, -0.0210, -0.0386],
        [ 0.0262,  0.0036,  0.0177,  ...,  0.0095,  0.0262,  0.0458],
        [-0.0075,  0.0222,  0.0095,  ..., -0.0252,  0.0078, -0.0473],
        ...,
        [ 0.0484, -0.0228,  0.0109,  ...,  0.0413,  0.0353,  0.0141],
        [-0.0573, -0.0264,  0.0384,  ..., -0.0171,  0.0509,  0.0092],
        [ 0.0014,  0.0378,  0.0547,  ...,  0.0561, -0.0164,  0.0293]],
       requires_grad=True), Parameter containing:
tensor([[-0.0223,  0.0408,  0.0353,  ..., -0.0111,  0.0082, -0.0200],
        [-0.0594,  0.0564, -0.0236,  ...,  0.0222, -0.0415,  0.0352],
        [-0.0141, -0.0103, -0.0269,  ..., -0.0026,  0.0456, -0.0060],
        ...,
        [-0.0317, -0.0594, -0.0159,  ...,  0.0096, -0.0624,  0.0097],
        [ 0.0369,  0.0284,  0.0323,  ..., -0.0006, -0.0616, -0.0105],
        [-0.0213,  0.0236,  0.0355,  ...,  0.0407,  0.0251,  0.0141]],
       requires_grad=True), Parameter containing:
tensor([[

In [ ]:
# Add your initial primer strings to this list for testing
sentences = ['time traveller',
             'traveller',
             'machine'

]

for sentence in sentences:
    print(predict(sentence, 50, model, vocab))

time traveller ffree if a mo in all dlancl this is all phased un
travelleryou can show black is white by argument said filby
machine i wo ch anguttee ote himentron puotion anot the t


Summarize the results of your ablation study. What changes did you make? How was the performance affacted? Provide reasoning for your observations. (200-400 words)

In [ ]:
section3_ans = """ 1. Xavier initialization of weights, standard LSTM,                             500 epoch, LR = 1   : Perplexity goes from ~17 --> 1.1. Output of test has words that make sense but many are nonsense, particularly at end of the sentence.
                   2. Xavier initialization of weights, standard LSTM,                             500 epoch, LR = 0.1 : Perplexity goes from ~25 --> 11.2. Output of test has mostly the word 'the'
                   3. Xavier initialization of weights, LSTM w/peepholes,                          500 epoch, LR = 1   : Perplexity goes from ~17 --> 1.0. Peepholes on all gates. Converged by ~250 epochs. Output of test has words that make sense but end of sentence gets sloppy. Lost spaces leading to combined words. Otherwise better than w/o peepholes
                   4. Xavier initialization of weights, LSTM w/coupled input and forget,           500 epoch, LR = 1   : Perplexity goes from ~18 --> 1.1. Output of test has words that make sense but end of sentence is not great. Also lost spaces but not too bad. 
                   5. Xavier initialization of weights, Changed LSTM to GRU,                       500 epoch, LR = 1   : Perplexity goes from ~16 --> 1.1. Output of test has words that make sense but in some cases things went wrong quickly. When a mistake was made early the rest of the sentence was no longer correct words.
                   6. Xavier initialization of weights, Changed LSTM to GRU w/dropout(0.5) b4 FCN, 500 epoch, LR = 1   : Perplexity goes from ~18 --> 1.8. Output of test has only a word or two at the beginning that make sense but otherwise are not very coherent. 
                   7. Xavier initialization of weights, Changed LSTM to GRU w/dropout(X,0.5),      500 epoch, LR = 1   : Perplexity goes from ~18 --> 8.1. Output of test is bad and gets stuck repeating words starting with 't', such as 'time','the','then','thoe','toe',etc """ #@param {type:"string"}

## Section 4: Know-Your-Pod Better

Discuss with two other members of your pod. What is their favorite movie scene (not just the whole movie, but describe the scene!) of all time? Describe the scene and why they like it so much. (~100 words each)

In [ ]:
section4_ans = 'Karan: end of inception where everything comes together. Abhisheck: Scene in children of men that is a neat action scene' #@param {type:"string"}

---
# Submission
Once you're done, click on 'Share' and add the link to the box below.


In [ ]:
import time
import numpy as np
import urllib.parse
from IPython.display import IFrame

t7 = time.time()

#@markdown #Run Cell to Show Airtable Form
#@markdown ##**Confirm your answers and then click "Submit"**


def prefill_form(src, fields: dict):
  '''
  src: the original src url to embed the form
  fields: a dictionary of field:value pairs,
  e.g. {"pennkey": my_pennkey, "location": my_location}
  '''
  prefill_fields = {}
  for key in fields:
      new_key = 'prefill_' + key
      prefill_fields[new_key] = fields[key]
  prefills = urllib.parse.urlencode(prefill_fields)
  src = src + prefills
  return src



#autofill fields if they are not present
#a missing pennkey and pod will result in an Airtable warning
#which is easily fixed user-side.
try: my_pennkey;
except NameError: my_pennkey = ""

try: my_pod;
except NameError: my_pod = "Select"

try: section1_ans;
except NameError: section1_ans = ""

try: section2_ans;
except NameError: section2_ans = ""

try: section3_ans;
except NameError: section3_ans = ""

try: section4_ans;
except NameError: section4_ans = ""


fields = {"pennkey": my_pennkey,
          "pod": my_pod,
          "section1_ans": section1_ans,
          "section2_ans": section2_ans,
          "section3_ans": section3_ans,
          "section4_ans": section4_ans}

src = "https://airtable.com/shrZdjQjhvUSHVgW5?"


#now instead of the original source url, we do: src = prefill_form(src, fields)
display(IFrame(src = prefill_form(src, fields), width = 800, height = 400))